## Pre-setup

In [ ]:
# import the aetest module
from pyats import aetest
import logging
from genie.testbed import load
import argparse
import sys
import pprint

# set debug level DEBUG, INFO, WARNING
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# set pprint
pp = pprint.PrettyPrinter(indent = 2)

## Time for the timestamps

In [83]:
import time

# %Y: full year (4 digits)
# %m: month (2 digits)
# %d: day (2 digits)
# %H: hours
# %M: minutes
# %S: seconds
# Doc: https://docs.python.org/fr/3/library/time.html#time.strftime

# I have to reuse these two lines every time I need the current_time.
t = time.localtime()
current_time = time.strftime("%Y-%m-%d %H:%M:%S", t)

print(current_time)

2020-11-17 11:11:53


## Loading the testbed, connecting to the device

In [3]:
testbed = load('/home/anorsoni/Projets/2020-CAP-Altitude/pyats-check-os/testbed.yaml')

asr903_5 = testbed.devices["ASR903_5"]

asr903_5.connect(init_exec_commands=[],
                 init_config_commands=[],
                 log_stdout=False)

In [4]:
isis = asr903_5.parse('show isis neighbors')
pp.pprint(isis)

{ 'isis': { 'Altitude': { 'neighbors': { 'ASR903_3': { 'type': { 'L2': { 'circuit_id': '1E',
                                                                         'holdtime': '25',
                                                                         'interface': 'BD1',
                                                                         'ip_address': '100.100.35.3',
                                                                         'state': 'UP'}}},
                                         'ASR903_4': { 'type': { 'L2': { 'circuit_id': '09',
                                                                         'holdtime': '25',
                                                                         'interface': 'Te0/2/4',
                                                                         'ip_address': '100.100.45.4',
                                                                         'state': 'UP'}}}}}}}


## Loading the DB

In [97]:
import sqlite3
db_connection = sqlite3.connect('../sqlite/db/checks.db')
db_cursor = db_connection.cursor()

## Setting up the DB


In [66]:
db_cursor.execute('''CREATE TABLE isis (MyRouter text, Neighbor text, Type text, Interface text, Ip text, Timestamp text)''')
db_cursor.execute('''CREATE TABLE timestamps (MyRouter text, TestName text, WhenTested text, TimeStamp text)''')
db_connection.commit()

## Getting isis information

In [116]:
my_router = asr903_5.name
when_tested = "before"

# Getting the current_time before the test
t = time.localtime()
current_time = time.strftime("%Y-%m-%d %H:%M:%S", t)

# Adding a line in the timestamps table
# | MyRouter | TestName | WhenTested | Timestamp   |
timestamps_tuple = []
timestamps_tuple.extend((my_router, "isis", when_tested, current_time))
db_cursor.execute('''INSERT INTO timestamps VALUES (?,?,?,?)''', timestamps_tuple)

for tag in isis['isis']:

    # I will create a line in the DB for each neighbor
    for neighbor in isis['isis'][tag]['neighbors']:
        neighbor = neighbor

        for isis_type in isis['isis'][tag]['neighbors'][neighbor]['type']:
            type = isis_type
            interface = isis['isis'][tag]['neighbors'][neighbor]['type'][type]['interface']
            ip_address = isis['isis'][tag]['neighbors'][neighbor]['type'][type]['ip_address']

        # Creating a isis_tuple to insert in the isis table.
        # | MyRouter | Neighbor | Type | Interface | Ip | Timestamp |
        isis_tuple = []
        isis_tuple.extend((my_router, neighbor, type, interface, ip_address, current_time))
    
        # Inserting a line for each neighbor in the 
        db_cursor.execute('''INSERT INTO isis VALUES (?,?,?,?,?,?)''', isis_tuple)

# Comitting the changes
db_connection.commit()

print("Done!")

Done!


## Fecthing data from a DB
### Fecthing from table isis

In [118]:
db_cursor.execute('''SELECT * FROM isis''')
for line in db_cursor.fetchall():
    print(line)
                    

('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3', '2020-11-17 11:47:12')
('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4', '2020-11-17 11:47:12')
('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3', '2020-11-17 11:51:45')
('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4', '2020-11-17 11:51:45')


### Fecthing from table timestamps

In [119]:
db_cursor.execute('''SELECT * FROM timestamps''')
for line in db_cursor.fetchall():
    print(line)

('ASR903_5', 'isis', 'before', '2020-11-17 11:47:12')
('ASR903_5', 'isis', 'before', '2020-11-17 11:51:45')
